In [1]:
import os
import re
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaModel, BertModel, AutoModel
from transformers import RobertaTokenizer, BertTokenizer, AutoTokenizer
import math
import time
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

%matplotlib inline

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: TITAN X (Pascal)


# Data Preprocessing

In [3]:
all_data = pd.read_csv('kaggle_data/train.csv')
test_data = pd.read_csv('kaggle_data/test.csv')
sample = pd.read_csv('kaggle_data/sample_submission.csv')

num_bins = int(np.floor(1 + np.log2(len(all_data))))
print(num_bins)
all_data.loc[:,'bins'] = pd.cut(all_data['target'],bins=num_bins,labels=False)

bins = all_data.bins.to_numpy()
target = all_data.target.to_numpy()

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

all_data.head()

12


,id,url_legal,license,excerpt,target,standard_error,bins
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,7
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,7
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,6
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,5
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,8


In [4]:
config = {
    'lr': 2e-5,
    'wd':0.01,
    'batch_size':16,
    'valid_step':10,
    'max_len':256,
    'epochs':3,
    'nfolds':5,
    'seed':42,
}


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])


In [5]:
all_data.head() #顯示前五筆數據

,id,url_legal,license,excerpt,target,standard_error,bins
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,7
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,7
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,6
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,5
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,8


In [6]:
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer,max_len=256, testing=False):
        self.excerpt = df['excerpt'].to_numpy()
        self.testing = testing
        if not self.testing:self.targets = df['target'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                #return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        if not self.testing:
            target = torch.tensor(self.targets[idx],dtype=torch.float) 
            return torch.tensor(encode.get('input_ids')), torch.tensor(encode.get('attention_mask')), target
        else:return torch.tensor(encode.get('input_ids')), torch.tensor(encode.get('attention_mask'))
    
    def __len__(self):
        return len(self.excerpt)

In [11]:
tmpDataset = CLRPDataset(all_data, tokenizer)
tmpIter = iter(tmpDataset)
token_ids, tmp_mask, target_score = next(tmpIter)
token_ids, tmp_mask = token_ids.unsqueeze(0), tmp_mask.unsqueeze(0)
token_ids, tmp_mask, target_score

(tensor([[    0,  1779,     5,   664,    82,  1835,     7,     5,  1011,  4294,
              6,    24,  2633,    10, 27265,  1714,  2772,     4,  2978,     9,
             41,  6291,  1310,     6,    24,    21,    10,  2608,  5252,     4,
          50118,   133,  1929,    21,  2913,    19,  1958,    12,  9830, 20790,
              6,    45,  4976,    15, 17359,     6,    53, 11122, 18331,    81,
          24271,     8,  9910,  6368,     6,   101,    10,   588,  1958,   882,
              4,    20,  3617, 38325,     8,   655,   571, 18656,    14,    56,
          14633,     5,   929,     6,    58, 39143,    19, 15039,     8, 22246,
          11538,    19, 13145, 21811,     9, 13178,     6,   101,  1958,     4,
           1578, 11720,  8402,    56,    57, 14998, 38073,    15,   106,     6,
              8, 19053,   154, 16155, 41591, 20846, 10601,    31,     5,  9836,
              4, 50118,  3750,   349,   253,     9,     5,   929,     6,    15,
              5,  2204,     6, 10601,   

In [12]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super(AttentionHead, self).__init__()
        self.W1 = nn.Linear(in_features, hidden_dim)
        #self.W2 = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)

    def forward(self, features):
        #query, key = self.W1(features), self.W2(features)
        att = torch.tanh(self.W1(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)
        return context_vector

In [17]:
%%time
import torch
import torch.nn as nn

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 64, 1

        # Instantiate BERT model
        #self.bert = BertModel.from_pretrained('bert-base-uncased')
        #self.bert = RobertaModel.from_pretrained('roberta-base')
        #self.bert = AutoModel.from_pretrained('google/electra-base-discriminator', output_hidden_states=True)
        self.bert = AutoModel.from_pretrained('roberta-base', output_hidden_states=True)
        #print(self.bert.config.hidden_size)
        self.head = AttentionHead(D_in, D_in, 1)
        
        #self.pooler = AttentionPooling(12, D_in, D_in)
        
        self.linear = nn.Linear(D_in, D_out)
        
        self.dropout = nn.Dropout(0.1)
        
        # Instantiate an one-layer feed-forward classifier
        self.cls = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        
        # Feed input to BERT
        
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)[0]
        outputs = self.head(outputs)
        outputs = self.dropout(outputs)
        logits = self.linear(outputs)
        '''
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        all_hidden_state = torch.stack(outputs[2])        
        
        #print(all_hidden_state[-3:].shape)
        last_hidden_state_cls = outputs[0][:, 0, :]
        last_hidden_state = outputs[0]
        outputs = self.pooler(all_hidden_state)
        logits = self.linear(outputs)  
        '''  

        return logits

Wall time: 0 ns


In [16]:
tmp_model = BertClassifier()
tmp_output = tmp_model(token_ids, tmp_mask)
tmp_output

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768


tensor([[-0.1594]], grad_fn=<AddmmBackward>)

In [9]:
from transformers import AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

def create_optimizer(model):
    named_parameters = list(model.named_parameters())    
    
    roberta_parameters = named_parameters[:197]    
    attention_parameters = named_parameters[199:203]
    regressor_parameters = named_parameters[203:]
        
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]

    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})

    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if "bias" in name else 0.01

        lr = 2e-5

        if layer_num >= 69:        
            lr = 5e-5

        if layer_num >= 133:
            lr = 1e-4

        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return AdamW(parameters)

In [10]:

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = create_optimizer(bert_classifier)
    '''
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )
    '''

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    '''
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    '''
    scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [11]:
import random
import time

# Specify loss function

def loss_fn(outputs,targets):
    outputs = outputs.view(-1)
    targets = targets.view(-1)
    return torch.sqrt(nn.MSELoss()(outputs,targets))
    
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            
            # Zero out any previously calculated gradients
            model.zero_grad()
            inputs, b_attn_mask, targets = tuple(t.to(device) for t in batch)
            
            # Perform a forward pass. This will return logits.
            logits = model(inputs, b_attn_mask)
            
            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, targets)
            batch_loss += loss.item()
            total_loss += loss.item()
            #print(loss)
            #input()
            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()
                
        

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)
        
        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [12]:
kfold = KFold(n_splits=config['nfolds'], random_state=config['seed'], shuffle=True)
for fold, (train_indices, val_indices) in enumerate(kfold.split(all_data)):   
    seed_everything(seed=config['seed']+fold)
    
    train_ds = CLRPDataset(all_data.loc[train_indices], tokenizer, config['max_len'])
    train_dataloader = DataLoader(train_ds,
                        batch_size = config["batch_size"],
                        shuffle=True,
                        num_workers = 0,
                        pin_memory=True,
                        drop_last=False)
    
    val_ds = CLRPDataset(all_data.loc[val_indices],tokenizer,config['max_len'])
    val_dataloader = DataLoader(val_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 0,
                        pin_memory=True,
                        drop_last=False)
    
    bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
    train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)
    #儲存模型
    torch.save(bert_classifier.state_dict(), 'ckpt/F_{}.h5'.format(fold+1))
    del bert_classifier
   

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.846740   |     -      |     -     |   8.77   
   1    |   40    |   0.665135   |     -      |     -     |   8.00   
   1    |   60    |   0.644878   |     -      |     -     |   8.13   
   1    |   80    |   0.625393   |     -      |     -     |   8.14   
   1    |   100   |   0.617421   |     -      |     -     |   8.17   
   1    |   120   |   0.552315   |     -      |     -     |   8.18   
   1    |   140   |   0.537852   |     -      |     -     |   8.18   
   1    |   141   |   0.535736   |     -      |     -     |   0.30   
----------------------------------------------------------------------
   1    |    -    |   0.642093   |  0.552916  |   0.00    |   62.84  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elaps

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.916889   |     -      |     -     |   8.70   
   1    |   40    |   0.616226   |     -      |     -     |   8.22   
   1    |   60    |   0.661362   |     -      |     -     |   8.18   
   1    |   80    |   0.637257   |     -      |     -     |   8.20   
   1    |   100   |   0.582819   |     -      |     -     |   8.17   
   1    |   120   |   0.568920   |     -      |     -     |   8.20   
   1    |   140   |   0.527992   |     -      |     -     |   8.20   
   1    |   141   |   0.569377   |     -      |     -     |   0.30   
----------------------------------------------------------------------
   1    |    -    |   0.645884   |  0.571873  |   0.00    |   63.03  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elaps

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.888024   |     -      |     -     |   8.73   
   1    |   40    |   0.702467   |     -      |     -     |   8.17   
   1    |   60    |   0.584682   |     -      |     -     |   8.24   
   1    |   80    |   0.584556   |     -      |     -     |   8.24   
   1    |   100   |   0.603364   |     -      |     -     |   8.35   
   1    |   120   |   0.608491   |     -      |     -     |   8.18   
   1    |   140   |   0.540467   |     -      |     -     |   8.19   
   1    |   141   |   0.515904   |     -      |     -     |   0.30   
----------------------------------------------------------------------
   1    |    -    |   0.645387   |  0.663819  |   0.00    |   63.56  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elaps

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.951897   |     -      |     -     |   8.67   
   1    |   40    |   0.710859   |     -      |     -     |   8.21   
   1    |   60    |   0.647648   |     -      |     -     |   8.17   
   1    |   80    |   0.571456   |     -      |     -     |   8.19   
   1    |   100   |   0.509084   |     -      |     -     |   8.19   
   1    |   120   |   0.569208   |     -      |     -     |   8.20   
   1    |   140   |   0.537091   |     -      |     -     |   8.20   
   1    |   141   |   0.369538   |     -      |     -     |   0.30   
----------------------------------------------------------------------
   1    |    -    |   0.642721   |  0.547651  |   0.17    |   63.06  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elaps

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.916636   |     -      |     -     |   8.77   
   1    |   40    |   0.710176   |     -      |     -     |   8.32   
   1    |   60    |   0.698506   |     -      |     -     |   8.48   
   1    |   80    |   0.755961   |     -      |     -     |   8.37   
   1    |   100   |   0.709238   |     -      |     -     |   8.17   
   1    |   120   |   0.582838   |     -      |     -     |   8.18   
   1    |   140   |   0.555405   |     -      |     -     |   8.20   
   1    |   141   |   0.495897   |     -      |     -     |   0.33   
----------------------------------------------------------------------
   1    |    -    |   0.704139   |  0.591193  |   0.00    |   63.91  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elaps

In [16]:
test_ds = CLRPDataset(test_data,tokenizer,config['max_len'],True)
test_dataloader = DataLoader(test_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 0,
                        pin_memory=True,
                        drop_last=False)

In [17]:
def get_prediction(test_dataloader, model_path, ckpt_path):        
    model = BertClassifier(freeze_bert=False)
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    model.to(device)
    model.eval()
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    predictions = list()
    for i, (batch) in tqdm(enumerate(test_dataloader)):
        inputs, b_attn_mask = tuple(t.to(device) for t in batch)
        outputs = model(inputs, b_attn_mask)
        outputs = outputs.cpu().detach().numpy().ravel().tolist()
        predictions.extend(outputs)
        
    torch.cuda.empty_cache()
    del model
    return np.array(predictions)

In [18]:
all_predictions = np.zeros((config['nfolds'], len(test_data)))
for index in range(config['nfolds']): 
    pred = get_prediction(test_dataloader, 'roberta-base', f'ckpt/F_{index+1}.h5')
    all_predictions[index] = pred

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
1it [00:00, 12.81it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are 

In [19]:
sample['target'] = all_predictions.mean(axis=0)
sample.to_csv('submission.csv',index=False)

In [20]:
sample

,id,target
0,c0f722661,-0.230035
1,f0953f0a5,-0.058294
2,0df072751,-0.102372
3,04caf4e0c,-2.388841
4,0e63f8bea,-1.435886
5,12537fe78,-1.059483
6,965e592c0,0.565299
